In [58]:
import sqlite3
import pandas as pd

#Employment Projection Import - Start
empProj = pd.read_excel("EmploymentProjections.xlsx", header=None, skiprows=3, skip_footer=4)
empProj.columns = ['Employment_Title', 'Matrix_Code', 'Occupation', 'Employment_2016', 'Employment_2026', 'Employment_Change_Number', 'Employment_Change_Percent', 'Self_Employed_Percent', 'Openings_2016_2026', 'Median_Wage', 'Education_Needed', 'Work_Experience', 'Job_Training']
empProj = empProj.drop(empProj[empProj.Occupation.str.contains("Summary")].index)
#print(empProj)
#Employment Projection Import - End

#Undergraduate Cost Import - Start
undCost = pd.read_excel("UndergraduateCost.xlsx", header=None, skiprows=6, skip_footer=3)
undCost = undCost.drop(undCost.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20]], axis=1)
undCost.columns = ['Category', 'All_Institutions_2016']
undCost = undCost.drop(undCost[undCost.Category.str.contains("Tuition")==False].index)
undCost = undCost.dropna(subset=['Category'])
undCost = undCost[:-2]
undCost.iloc[-2, undCost.columns.get_loc('Category')] = "4-year institutions"
undCost.iloc[-1, undCost.columns.get_loc('Category')] = "2-year institutions"
#print(undCost)
#Undergraduate Cost Import - End

#Graduate Cost Import - Start
gradCost = pd.read_excel("GraduateCost.xlsx", header=None, skiprows=5, skip_footer=4)
gradCost = gradCost.drop(gradCost.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1)
gradCost.columns = ['Year', 'All_Institutions']
gradCost = gradCost.dropna(subset=['Year'])
gradCost = gradCost[:-27]

#Graduate Education Cost Change - For Doctoral Data - Start
eduGrad_2009 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2008-09")].index[0], 'All_Institutions')
eduGrad_2010 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2009-10")].index[0], 'All_Institutions')
eduGrad_2011 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2010-11")].index[0], 'All_Institutions')
eduGrad_2012 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2011-12")].index[0], 'All_Institutions')
eduGrad_2013 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2012-13")].index[0], 'All_Institutions')
eduGrad_2014 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2013-14")].index[0], 'All_Institutions')
eduGrad_2015 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2014-15")].index[0], 'All_Institutions')
eduGrad_2016 = gradCost.get_value(gradCost[gradCost.Year.str.contains("2015-16")].index[0], 'All_Institutions')

eduChange = (eduGrad_2010 / eduGrad_2009) * (eduGrad_2011 / eduGrad_2010) * (eduGrad_2012 / eduGrad_2011) * (eduGrad_2013 / eduGrad_2012) * (eduGrad_2014 / eduGrad_2013) * (eduGrad_2015 / eduGrad_2014) * (eduGrad_2016 / eduGrad_2015)
#Graduate Education Cost Change - For Doctoral Data - End

gradCost = gradCost.drop(gradCost[gradCost.Year.str.contains("2015-16")==False].index)
gradCost.iloc[-1, gradCost.columns.get_loc('Year')] = "2016"
#print(gradCost)
#Graduate Cost Import - End

#Doctoral Cost Import - Start
docCost = pd.read_excel("DoctoralCost.xlsx", header=None, skiprows=5, skip_footer=6)
docCost = docCost.drop(docCost.columns[[2]], axis=1)
docCost.columns = ['Year','Average','Chiropractic', 'Dentistry','Medicine','Optometry','Osteopathic_Medicine','Pharmacy','Podiatry','Veterinary','Law','Theology']
docCost = docCost.drop(docCost[docCost.Year.str.contains("2008-09")==False].index)
docCost = docCost.dropna(subset=['Year'])
docCost = docCost[:-2]

#Doctoral Education Cost Change - Using Graduate Education Change - Start
docCost = docCost[['Average','Chiropractic', 'Dentistry','Medicine','Optometry','Osteopathic_Medicine','Pharmacy','Podiatry','Veterinary','Law','Theology']].multiply(eduChange, axis="index")
docCost.insert(loc=0, column='Year', value=['2016'])
#Doctoral Education Cost Change - Using Graduate Education Change - End
    
#print(docCost)
#Doctoral Cost Import - End


conn = sqlite3.connect('employment_analysis.db')
cur = conn.cursor() 

empProj.to_sql('employmentProjection', conn, if_exists='replace')
undCost.to_sql('undergraduateCost', conn, if_exists='replace')
gradCost.to_sql('graduateCost', conn, if_exists='replace')
docCost.to_sql('doctoralCost', conn, if_exists='replace')

cur.close()
conn.close()



In [66]:
conn = sqlite3.connect('employment_analysis.db')
cur = conn.cursor()

empProjresults = cur.execute('SELECT * FROM employmentProjection LIMIT 3;')

for row in empProjresults.fetchall():
    print(row)
    
undCostresults = cur.execute('SELECT * FROM undergraduateCost LIMIT 2;')

for row in undCostresults.fetchall():
    print(row)
    
gradCostresults = cur.execute('SELECT * FROM graduateCost LIMIT 1;')

for row in gradCostresults.fetchall():
    print(row)
    
docCostresults = cur.execute('SELECT * FROM doctoralCost LIMIT 1;')

for row in docCostresults.fetchall():
    print(row)

#test = pd.read_sql('SELECT * FROM doctoralCost LIMIT 1;', conn)
#print(test)

testresults = cur.execute('SELECT Education_Needed FROM employmentProjection GROUP BY Education_Needed;')

for row in testresults.fetchall():
    print(row)

cur.close()
conn.close()


(3, 'Chief executives', '11-1011', 'Line item', 308.9, 296.8, -12.1, -3.9, '22.8', 20.0, '181210', "Bachelor's degree", '5 years or more', 'None')
(4, 'General and operations managers', '11-1021', 'Line item', 2263.1, 2468.3, 205.2, 9.1, '0.6', 210.7, '99310', "Bachelor's degree", '5 years or more', 'None')
(5, 'Legislators', '11-1031', 'Line item', 55.5, 59.4, 3.9, 7.1, '—', 4.4, '23470', "Bachelor's degree", 'Less than 5 years', 'None')
(7, '4-year institutions', 17181.804648322104)
(26, '2-year institutions', 4161.47898169657)
(31, '2016', 17868.480850081924)
(24, '2016', 17851.38813514861, 31396.353327973244, 49156.48805110744, 39135.4427697134, 28968.584496728883, 44002.432590847355, 27289.27521891786, 35506.36072265488, 23860.073331035073, 38736.54511165488, 15689.850264717597)
("Associate's degree",)
("Bachelor's degree",)
('Doctoral or professional degree',)
('High school diploma or equivalent',)
("Master's degree",)
('No formal educational credential',)
('Postsecondary nondegr